Import libraries

In [24]:
import pandas as pd
from datetime import datetime
import os

## Step 1: Load data from URL

In [29]:
def load_excel_by_column_names(url, engine=None, expected_start=("boro", "block", "lot"), max_scan=30):
    """
    Loads an Excel file starting from the first row where column headers match `expected_start`.
    """
    temp = pd.read_excel(url, header=None, engine=engine)

    for i in range(max_scan):
        row = temp.iloc[i].astype(str).str.strip().str.lower().tolist()
        if row[:len(expected_start)] == list(expected_start):
            print(f"✅ Header row found at line {i - 1}")
            return pd.read_excel(url, skiprows=i, engine=engine)

    raise ValueError("❌ Header row not found within scan range.")

<b> Use this (change file path) if it is an xlsx file </b>

In [26]:
df_2025 = load_excel_by_column_names(
    "https://www.nyc.gov/assets/finance/downloads/pdf/nopv/revised/revised_nopv-05142025.xlsx",
    engine="openpyxl"
)

✅ Header row found at line 10


<b> Use this (change file path) if xls file </b>

In [30]:
df_2022 = load_excel_by_column_names(
    "https://www.nyc.gov/assets/finance/downloads/pdf/nopv/revised/revised-nopv-05182022.xls",
    engine="xlrd"
)

✅ Header row found at line 11


<b> Check </b>

In [31]:
df_2022
print(df_2022.columns)

Index(['BORO', 'BLOCK', 'LOT', 'EASE', 'ADDRESS 1', 'ADDRESS 2', 'ADDRESS 3',
       'CITY, STATE, ZIP', 'COUNTRY', 'Unnamed: 9', 'Unnamed: 10', 'ORIGINAL',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'REVISED', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'RC1', 'RC2', 'RC3', 'RC4', 'RC5'],
      dtype='object')


# Step 2: Make clean function

In [21]:
def clean_nopv(df, fiscal_year, publish_date):
    """
    Cleans a Notice of Property Value DataFrame by standardizing structure and adding key fields.
    """
    # 1. Normalize column names
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

    # 2. Drop rows without core identifying info
    df = df.dropna(subset=["boro", "block", "lot"])

    # 3. Rename final roll value columns if they exist
    rename_map = {
        "market_value.1": "market_value_final_roll",
        "taxable_value.1": "taxable_value_final_roll",
        "transitional_assesssed_value.1": "transitional_assessed_value_final_roll",
        "transitional_exemption.1": "transitional_exemption_final_roll"
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    # 4. Create BBL
    df["bbl"] = (
        df["boro"].astype(str).str.zfill(1) +
        df["block"].astype(str).str.zfill(5) +
        df["lot"].astype(str).str.zfill(4)
    )

    # 5. Add publication metadata
    df["fiscal_year"] = fiscal_year
    df["date_published"] = publish_date
    df["published_year"] = publish_date.year
    df["published_month"] = publish_date.month
    df["published_day"] = publish_date.day

    return df


In [22]:
from datetime import datetime

df_2025_raw = load_excel_by_column_names(
    "https://www.nyc.gov/assets/finance/downloads/pdf/nopv/revised/revised_nopv-05142025.xlsx",
    engine="openpyxl"
)

df_2025 = clean_nopv(df_2025_raw, fiscal_year=2025, publish_date=datetime(2025, 5, 14))
df_2025.head()


✅ Header row found at line 10


,boro,block,lot,ease,address_1,address_2,address_3,"city,_state,_zip",country,tax_class,...,rc2,rc3,rc4,rc5,bbl,fiscal_year,date_published,published_year,published_month,published_day
0,1,11,14,NaN,NaN,855 AVENUE OF THE AMERIC,NaN,NEW YORK NY 10001-4105,NaN,2,...,NaN,NaN,NaN,NaN,1000110014,2025,2025-05-14,2025,5,14
1,1,15,1004,NaN,NaN,1 S BOULEVARD,NaN,NYACK NY 10960-3604,NaN,4,...,NaN,NaN,NaN,NaN,1000151004,2025,2025-05-14,2025,5,14
2,1,18,1134,NaN,NaN,88 GREENWICH ST,APT 718,NEW YORK NY 10006-2229,NaN,2,...,NaN,NaN,NaN,NaN,1000181134,2025,2025-05-14,2025,5,14
3,1,18,1160,NaN,NaN,88 GREENWICH ST,APT 815,NEW YORK NY 10006-2231,NaN,2,...,NaN,NaN,NaN,NaN,1000181160,2025,2025-05-14,2025,5,14
4,1,26,1008,NaN,NaN,55 EXCHANGE PL,NaN,NEW YORK NY 10005-3301,NaN,4,...,NaN,NaN,NaN,NaN,1000261008,2025,2025-05-14,2025,5,14


In [23]:
print(df_2025.columns)

Index(['boro', 'block', 'lot', 'ease', 'address_1', 'address_2', 'address_3',
       'city,_state,_zip', 'country', 'tax_class', 'bldg_class',
       'market_value', 'assessed_value', 'exemption',
       'transitional_assesssed_value', 'transitional_exemption',
       'taxable_value', 'market_value_final_roll',
       'transitional_assessed_value_final_roll', 'exemption.1',
       'transitional_assesssed_value.2', 'transitional_exemption_final_roll',
       'taxable_value_final_roll', 'rc1', 'rc2', 'rc3', 'rc4', 'rc5', 'bbl',
       'fiscal_year', 'date_published', 'published_year', 'published_month',
       'published_day'],
      dtype='object')
